In [1]:
import gurobipy as gp
from gurobipy import *
import numpy as np
import pandas as pd

In [2]:
# Dictionaries for factories (supply in tons), depots(throughput in tons), customers(demand in tons)
f = dict({
    'Liverpool': 150000,
    'Brighton': 200000
})

d = dict({
    'Newcastle': 70000,
    'Birmingham': 50000,
    'London': 100000,
    'Exeter': 40000
})

c = dict({
    'C1': 50000,
    'C2': 10000,
    'C3': 40000,
    'C4': 35000,
    'C5': 60000,
    'C6': 20000
})

In [3]:
# Dictionary for distribution costs per factory (in €/ton)
arcs, distr_cost = gp.multidict({
    ('Liverpool', 'Newcastle'): 0.5,
    ('Liverpool', 'Birmingham'): 0.5,
    ('Liverpool', 'London'): 1.0,
    ('Liverpool', 'Exeter'): 0.2,
    ('Liverpool', 'C1'): 1.0,
    ('Liverpool', 'C3'): 1.5,
    ('Liverpool', 'C4'): 2.0,
    ('Liverpool', 'C6'): 2.5,
    ('Brighton', 'Birmingham'): 0.3,
    ('Brighton', 'London'): 0.5,
    ('Brighton', 'Exeter'): 0.2,
    ('Newcastle', 'C2'): 1.5,
    ('Newcastle', 'C3'): 0.5,
    ('Newcastle', 'C5'): 3.0,
    ('Newcastle', 'C6'): 2.5,
    ('Birmingham', 'C2'): 2.0,
    ('Birmingham', 'C3'): 0.5,
    ('Birmingham', 'C4'): 1.0,
    ('Birmingham', 'C5'): 0.5,
    ('London', 'C2'): 3.0,
    ('London', 'C3'): 2.0,
    ('London', 'C5'): 2.0,
    ('London', 'C6'): 1.5,
    ('Exeter', 'C3'): 0.2,
    ('Exeter', 'C4'): 1.5,
    ('Exeter', 'C5'): 2.0,
    ('Exeter', 'C6'): 1.5    
})

In [4]:
# Define model
model = Model('Transshipment_Assignment')

# Create decision variables
plan = model.addVars(arcs, name="plan")

# set objective function
model.setObjective(plan.prod(distr_cost), GRB.MINIMIZE)

Academic license - for non-commercial use only - expires 2021-06-13
Using license file C:\Users\PC\gurobi.lic


In [5]:
# Define constraints
# maximal factory supply
factories = f.keys()
factory_con = model.addConstrs((gp.quicksum(plan.select(i, '*')) <= f[i] for i in factories), name="factory_constraint")

# maximal depot throughput
depots = d.keys()
depot_through_con = model.addConstrs((gp.quicksum(plan.select(j, '*')) <= d[j] for j in depots), 
                                     name="depot_through_constraint")
# depot output == depot input
depot_out_con = model.addConstrs((gp.quicksum(plan.select('*', j)) == gp.quicksum(plan.select(j, '*')) for j in depots),
                                                                                 name="depot_out_constraint")
# customer requirement == customer input
customers = c.keys()
customer_con = model.addConstrs((gp.quicksum(plan.select('*', k)) == c[k] for k in customers), name="customer_constraint")

In [6]:
model.optimize()

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 16 rows, 29 columns and 75 nonzeros
Model fingerprint: 0xec790327
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e-01, 4e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+04, 2e+05]
Presolve removed 1 rows and 1 columns
Presolve time: 0.03s
Presolved: 15 rows, 28 columns, 73 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.7300000e+05   2.624375e+04   0.000000e+00      0s
       7    2.6100000e+05   0.000000e+00   0.000000e+00      0s

Solved in 7 iterations and 0.05 seconds
Optimal objective  2.610000000e+05


In [7]:
# Optimal total distribution costs per month (in €)
obj = model.getObjective()
print("Total distribution costs per month: {0} €".format(obj.getValue()))

Total distribution costs per month: 261000.0 €


In [8]:
# Optimal monthly transportation plan in tons
# Extract quantities into a separate list
quantities=[]
for i in arcs:
    quantities.append(plan[i].x)
quantities

# Extract arcs into a separate list
arcs_list=list(arcs)

# Define a dataframe for arcs
df = pd.DataFrame(arcs_list, columns=["Supplier", "Receiver"])

# Add column for quantities
df["Quantity (in tons)"]=quantities

# Styling of dataframe
zero_columns=df[df["Quantity (in tons)"] == 0].index
df.drop(zero_columns, inplace=True)
trans_plan = df.round(2)
trans_plan

,Supplier,Receiver,Quantity (in tons)
0,Liverpool,Newcastle,10000.0
3,Liverpool,Exeter,40000.0
4,Liverpool,C1,50000.0
6,Liverpool,C4,35000.0
8,Brighton,Birmingham,50000.0
9,Brighton,London,30000.0
12,Newcastle,C2,10000.0
20,Birmingham,C5,50000.0
23,London,C5,10000.0
24,London,C6,20000.0
